In [1]:
###############################################################################
### pthon imports                                                           ###
###############################################################################

from os import path
from requests import get
from bs4 import BeautifulSoup
import os
import pandas as pd
import re

In [2]:
###############################################################################
### local imports                                                           ###
###############################################################################

import zachquire as zaq

In [3]:
def find_file(
    file_name='path/file.txt',
    cache=False,
    cache_age=None
    ):

    if cache==False:
        return None
    
    return path.exists(file_name)

In [4]:
def get_soup(
    url='https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    file_name='soupfiles/gitsearch.txt',
    cache=False,
    cache_age=None,
    slurper='*'
):
    '''
    
    '''
    # if we already have the data, read it locally
    file_found = find_file(file_name=file_name, cache=cache, cache_age=cache_age)
    if file_found:
        with open(file_name) as f:
            return True, BeautifulSoup(f.read())

    # otherwise go fetch the data
    response = get(url, headers=headers)
    soup = BeautifulSoup(response.text)
    slurps = soup.select(slurper)
    if len(slurps) == 0:
        print('***** NO DATA RETURNED *****')
        return False, soup
    
    # save it for next time
    with open(file_name, 'w') as f:
        f.write(str(slurps[0]))
        if len(slurps)>1:
            for slurp in slurps[1:]:
#                 print(str(slurp))
                f.write('\n' + str(slurp))
        
    with open(file_name) as f:    
        soup = BeautifulSoup(f.read())
#     pd.to_csv(slurps, header=None, index=False)

    return True, soup

In [5]:
slurped, soup = get_soup(
    url='https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories',
    cache=True,
    slurper='.repo-list-item a',
    file_name='soupfiles/gitsearch.txt'
)

In [6]:
def soup_loop_gitsearch(
    df,
    page_beg=1,
    page_end=20,
    url='https://github.com/',
    adder_prepage='search?o=desc&',
    adder_postpage='&q=advent+of+code&s=stars&type=Repositories',
    headers={'User-Agent': 'Nothing suspicious'},
    directory='soupfiles/',
    file_name='gitsearch',
    file_suffix='.txt',
    cache=False,
    slurper='.repo-list-item a',
):
    for page in range(page_beg, page_end+1):
        use_page = str(page)
        use_url = f'{url}{adder_prepage}p={use_page}{adder_postpage}'
        use_file = f'{directory}{file_name}{use_page:>02s}{file_suffix}'
        print(f'use_page: {use_page:>2s}')
        print(f'use_url: {use_url}')
        print(f'use_file: {use_file}')
        slurped, soup = get_soup(
            url = use_url,
            file_name = use_file,
            cache = cache,
            slurper = slurper,
        )
        log_file = use_file if slurped else None
        loopdict = {
            'soup': [soup], 
            'page': [page], 
            'url': [use_url], 
            'file_name': [log_file]
        }
        df = df.append(pd.DataFrame.from_dict(loopdict), ignore_index=True)
    return df
        

In [7]:
soup_df = pd.DataFrame([], columns=['soup','page','url','file_name'])
# df
soup_df = soup_loop_gitsearch(soup_df, cache=True, page_beg=1, page_end=20)

use_page:  1
use_url: https://github.com/search?o=desc&p=1&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch01.txt
use_page:  2
use_url: https://github.com/search?o=desc&p=2&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch02.txt
use_page:  3
use_url: https://github.com/search?o=desc&p=3&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch03.txt
use_page:  4
use_url: https://github.com/search?o=desc&p=4&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch04.txt
use_page:  5
use_url: https://github.com/search?o=desc&p=5&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch05.txt
use_page:  6
use_url: https://github.com/search?o=desc&p=6&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch06.txt
use_page:  7
use_url: https://github.com/search?o=desc&p=7&q=advent+of+code&s=stars&type=Repositories
use_file: soupfiles/gitsearch07.txt
use_page:  8
use_url: https://gith

In [8]:
soup_df

,soup,page,url,file_name
0,"<html><body><a class=""v-align-middle"" data-hyd...",1,https://github.com/search?o=desc&p=1&q=advent+...,soupfiles/gitsearch01.txt
1,"<html><body><a class=""v-align-middle"" data-hyd...",2,https://github.com/search?o=desc&p=2&q=advent+...,soupfiles/gitsearch02.txt
2,"<html><body><a class=""v-align-middle"" data-hyd...",3,https://github.com/search?o=desc&p=3&q=advent+...,soupfiles/gitsearch03.txt
3,"<html><body><a class=""v-align-middle"" data-hyd...",4,https://github.com/search?o=desc&p=4&q=advent+...,soupfiles/gitsearch04.txt
4,"<html><body><a class=""v-align-middle"" data-hyd...",5,https://github.com/search?o=desc&p=5&q=advent+...,soupfiles/gitsearch05.txt
5,"<html><body><a class=""v-align-middle"" data-hyd...",6,https://github.com/search?o=desc&p=6&q=advent+...,soupfiles/gitsearch06.txt
6,"<html><body><a class=""v-align-middle"" data-hyd...",7,https://github.com/search?o=desc&p=7&q=advent+...,soupfiles/gitsearch07.txt
7,"<html><body><a class=""v-align-middle"" data-hyd...",8,https://github.com/search?o=desc&p=8&q=advent+...,soupfiles/gitsearch08.txt
8,"<html><body><a class=""v-align-middle"" data-hyd...",9,https://github.com/search?o=desc&p=9&q=advent+...,soupfiles/gitsearch09.txt
9,"<html><body><a class=""v-align-middle"" data-hyd...",10,https://github.com/search?o=desc&p=10&q=advent...,soupfiles/gitsearch10.txt


In [9]:
print(soup_df.soup[19])

<html><body><a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":20,"per_page":10,"query":"advent of code","result_position":1,"click_id":47785268,"result":{"id":47785268,"global_relay_id":"MDEwOlJlcG9zaXRvcnk0Nzc4NTI2OA==","model_name":"Repository","url":"https://github.com/kodsnack/advent_of_code_2015"},"client_id":null,"originating_request_id":"C7F4:2C7D:5EBEA74:9ED209C:5DF038B4","originating_url":"https://github.com/search?o=desc&amp;p=20&amp;q=advent+of+code&amp;s=stars&amp;type=Repositories","referrer":null,"user_id":null}}' data-hydro-click-hmac="9da75b1cde3ca6cefec42158aacd558eb5da021c451d01da62dd3d92bfe514b2" href="/kodsnack/advent_of_code_2015">kodsnack/<em>advent</em>_<em>of</em>_<em>code</em>_2015</a>
<a href="http://adventofcode.com" rel="nofollow">http://adventofcode.com</a>
<a class="muted-link" href="/kodsnack/advent_of_code_2015/stargazers">
<svg aria-label="star" class="octicon octicon-star" height="16" role="img" ve

In [10]:
re_url = re.compile(r'"url"\:"(.+?)"')
urllist = re_url.findall(str(soup_df.soup[0]))

urllist

['https://github.com/Bogdanp/awesome-advent-of-code',
 'https://github.com/BurntSushi/advent-of-code',
 'https://github.com/Lysxia/advent-of-coq-2018',
 'https://github.com/fogleman/AdventOfCode2018',
 'https://github.com/mstksg/advent-of-code-2018',
 'https://github.com/Voltara/advent2018-fast',
 'https://github.com/wimglenn/advent-of-code-data',
 'https://github.com/ChrisPenner/Advent-Of-Code-Polyglot',
 'https://github.com/jaksi/advent-of-other-peoples-code',
 'https://github.com/molyswu/hand_detection']

In [11]:
def get_repos_from_gitsearch(df, column='soup', reg_text=r'"url"\:"(.+?)"'):
    git_urls = []
    re_url = re.compile(reg_text)
    for soup in soup_df.soup:
        git_urls.extend(re_url.findall(str(soup)))
    return list(set(git_urls))
        

In [12]:
url_list = get_repos_from_gitsearch(soup_df)

In [13]:
len(url_list)

197

In [14]:
# souprepo = get_soup(
#     url = 'https://github.com/mfikes/advent-of-code',
#     file_name = 'soupfiles/mfikes_advent-of-code.html',
#     slurper='html',
#     cache = False
# )


In [15]:
# souprepo = get_soup(
#     url = 'https://github.com/Diggsey/aoc2018',
#     file_name = 'soupfiles/diggsey_aoc2018.html',
#     slurper='html',
#     cache = False
# )

In [16]:
# souprepo = get_soup(
#     url = 'https://github.com/mhanberg/advent-of-code-elixir-starter',
#     file_name = 'soupfiles/mhanberg_advent-of-code-elixir-starter.html',
#     slurper='html',
#     cache = False
# )

In [17]:
# Library imports for sending request, parsing HTML, and pandas
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
# Example list of URLs from GitHub
urls = [
    "https://github.com/mfikes/advent-of-code",
    "https://github.com/ryanorsinger/intro-to-machine-learning-workshop",
    'https://github.com/Diggsey/aoc2018',
    "https://github.com/kjeliasen/AdventOfCode",
    'https://github.com/mhanberg/advent-of-code-elixir-starter'
]
output = []
for url in urls:
    request = get(url)
    soup = BeautifulSoup(request.text)
    title = url.split("/")[-1]
    author = url.split("/")[-2]
    language_info = soup.select_one(".repository-lang-stats-numbers")
    languages = [language.get_text() for language in language_info.select(".lang")]
    percentages = [percentage.get_text() for percentage in language_info.select(".percent")]
    readme = soup.select_one("#readme article")
    repo_data = {}
    repo_data["author"] = author
    repo_data["title"] = title
    repo_data["readme"] = readme
    repo_data["languages"] = languages
    repo_data["language_percentages"] = percentages
    output.append(repo_data)
df = pd.DataFrame(output)
df.to_csv("soupfiles/all_repo_data.csv")

In [18]:
df

,author,language_percentages,languages,readme,title
0,mfikes,[100.0%],[Clojure],"<article class=""markdown-body entry-content p-...",advent-of-code
1,ryanorsinger,[100.0%],[Jupyter Notebook],"<article class=""markdown-body entry-content p-...",intro-to-machine-learning-workshop
2,Diggsey,[100.0%],[Rust],None,aoc2018
3,kjeliasen,"[97.8%, 2.2%]","[Jupyter Notebook, Python]","<article class=""markdown-body entry-content p-...",AdventOfCode
4,mhanberg,[100.0%],[Elixir],"<article class=""markdown-body entry-content p-...",advent-of-code-elixir-starter


In [19]:
re_repo = re.compile(r'https\://github.com/(.+?)$')
repo_list = []
repo_check = [re_repo.findall(url)[0] for url in url_list]
repo_list.extend(repo_check)
repo_list

['BartoszMilewski/AdventOfCode18',
 'mstksg/advent-of-code-2019',
 'bewuethr/advent-of-code',
 'CameronAavik/AdventOfCode',
 'BartoszMilewski/AofC2017',
 'badouralix/advent-of-code-2018',
 'bildzeitung/2016adventofcode',
 'AlexeSimon/adventofcode',
 'unleashed-coding/advent-of-code',
 'willkill07/AdventOfCode2015',
 'tginsberg/advent-2019-kotlin',
 'wimglenn/advent-of-code-wim',
 'poteto/advent-of-code-2018',
 'maislinger/advent_of_code',
 'MitjaBezensek/advent-of-code',
 'molyswu/hand_detection',
 'jchristman/adventofcode',
 'moesol/pony-advent-2016',
 'ttencate/aoc2016',
 'njoubert/RaspberryPiChristmasCodingAdventure',
 'BartMassey/advent-of-code-2016',
 'Harleqin/advent-of-code-2017',
 'kodsnack/advent_of_code_2018',
 'Lokeh/advent-2019',
 'Vzaa/advent_of_code_2019',
 'kodsnack/advent_of_code_2016',
 'taylorwood/advent-of-code',
 'Lysxia/advent-of-coq-2018',
 'Dementophobia/advent-of-code-2019',
 'gchan/advent-of-code-ruby',
 'themattchan/advent-of-code',
 'MicUurloon/AdventurePathf

In [20]:
filepath = "datafiles/data.json"
zaq.scrape_github_data(repos=repo_list, filepath=filepath)

BartoszMilewski/AdventOfCode18
https://raw.githubusercontent.com/BartoszMilewski/AdventOfCode18/master/README.md
mstksg/advent-of-code-2019
https://raw.githubusercontent.com/mstksg/advent-of-code-2019/master/README.md
bewuethr/advent-of-code
https://raw.githubusercontent.com/bewuethr/advent-of-code/master/README.md
CameronAavik/AdventOfCode
https://raw.githubusercontent.com/CameronAavik/AdventOfCode/master/README.MD
BartoszMilewski/AofC2017
https://raw.githubusercontent.com/BartoszMilewski/AofC2017/master/README.md
badouralix/advent-of-code-2018
https://raw.githubusercontent.com/badouralix/advent-of-code-2018/master/README.md
bildzeitung/2016adventofcode
https://raw.githubusercontent.com/bildzeitung/2016adventofcode/master/README.md
AlexeSimon/adventofcode
https://raw.githubusercontent.com/AlexeSimon/adventofcode/master/README.md
unleashed-coding/advent-of-code
https://raw.githubusercontent.com/unleashed-coding/advent-of-code/master/README.md
willkill07/AdventOfCode2015
https://raw.git

https://raw.githubusercontent.com/cls1991/advent-of-code/master/README.md
nickbclifford/advent-of-code-2018
https://raw.githubusercontent.com/nickbclifford/advent-of-code-2018/master/README.md
narimiran/AdventOfCode2017
https://raw.githubusercontent.com/narimiran/AdventOfCode2017/master/README.md
NiXXeD/adventofcode
https://raw.githubusercontent.com/NiXXeD/adventofcode/master/README.md
encse/adventofcode
https://raw.githubusercontent.com/encse/adventofcode/master/README.md
jennybc/2018_advent-of-code
None
mstksg/advent-of-code-2018
https://raw.githubusercontent.com/mstksg/advent-of-code-2018/master/README.md
danthedaniel/Advent-of-Code-2017
https://raw.githubusercontent.com/danthedaniel/Advent-of-Code-2017/master/README.md
lynn/aoc-2019
https://raw.githubusercontent.com/lynn/aoc-2019/master/README.md
afonsojramos/advent-of-code-2018
https://raw.githubusercontent.com/afonsojramos/advent-of-code-2018/master/README.md
glguy/advent2017
https://raw.githubusercontent.com/glguy/advent2017/mas

https://raw.githubusercontent.com/mariotacke/advent-of-code-2018/master/README.md
glguy/advent2019
https://raw.githubusercontent.com/glguy/advent2019/master/README.md
ephemient/aoc2018
https://raw.githubusercontent.com/ephemient/aoc2018/master/README.md
believer/advent-of-code
https://raw.githubusercontent.com/believer/advent-of-code/master/README.md
mhanberg/advent-of-code-elixir-starter
https://raw.githubusercontent.com/mhanberg/advent-of-code-elixir-starter/master/README.md
dzikoysk/advent-of-code
https://raw.githubusercontent.com/dzikoysk/advent-of-code/master/README.md
KrzaQ/AdventOfCode2015
https://raw.githubusercontent.com/KrzaQ/AdventOfCode2015/master/README.md
joelgrus/advent2019
https://raw.githubusercontent.com/joelgrus/advent2019/master/README.md
Sketch-sh/advent-of-code-2018
https://raw.githubusercontent.com/Sketch-sh/advent-of-code-2018/master/README.md
BenoitZugmeyer/RustyAdventOfCode
https://raw.githubusercontent.com/BenoitZugmeyer/RustyAdventOfCode/master/README.md
Krz

In [57]:
git_json = pd.read_json(filepath).rename(columns={'repo':'repo_full'}).dropna().reset_index().drop(columns='index')
git_json['author'] = git_json.repo_full.apply(lambda x: x.split('/')[0])
git_json['repo'] = git_json.repo_full.apply(lambda x: x.split('/')[-1])
display(git_json.head())
git_json.info()

,language,language_all,readme_contents,repo_full,author,repo
0,Haskell,{'Haskell': 4468},# AdventOfCode18\nSolutions to some of the adv...,BartoszMilewski/AdventOfCode18,BartoszMilewski,AdventOfCode18
1,Haskell,{'Haskell': 139842},Advent of Code 2019\n=====================\n\n...,mstksg/advent-of-code-2019,mstksg,advent-of-code-2019
2,Perl,"{'Perl': 138429, 'Awk': 60552, 'Shell': 59915,...",# Advent of Code\n\nMy solutions to [Advent of...,bewuethr/advent-of-code,bewuethr,advent-of-code
3,F#,{'F#': 226573},# Advent of Code\n\nThese are my solutions in ...,CameronAavik/AdventOfCode,CameronAavik,AdventOfCode
4,Haskell,{'Haskell': 14869},# AofC2017\nAdvent of Code 2017\n,BartoszMilewski/AofC2017,BartoszMilewski,AofC2017


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 6 columns):
language           170 non-null object
language_all       170 non-null object
readme_contents    170 non-null object
repo_full          170 non-null object
author             170 non-null object
repo               170 non-null object
dtypes: object(6)
memory usage: 8.0+ KB


In [58]:
git_json.repo.value_counts().head()

advent-of-code         21
advent-of-code-2018    11
AdventOfCode2018        8
adventofcode            7
advent-of-code-2019     5
Name: repo, dtype: int64

In [59]:
mask = ['namen' in str(x.lower()) for x in git_json.readme_contents]
idx = git_json[mask].index
for rc in git_json.readme_contents.loc[idx]:
    print(rc)

# scy’s Advent of Code

👇 *Nach dem ersten Abschnitt geht’s komplett auf Deutsch weiter!* 👇

## Information in English

This repository contains my solutions to some [Advent of Code](https://adventofcode.com/) puzzles.
I’m using Advent of Code to learn new programming languages or to improve my proficiency in those I already know.

Also, each year in December, when the puzzles come out, I’m doing live video streams while solving them.
That way, people can look over my shoulder, ask questions or suggest other ways to build the solution.

While the code I’m writing uses English names and comments, **the streams are in German**, my mother tongue.
This allows me to think and speak more fluently, but more importantly it provides a way for German speakers who are not very good at English to follow along.
In other words, I’m trying to lower the entry barrier for people who struggle with English.

Therefore, the documentation and announcements in this repository that deal with the streams is i

In [60]:
# repo = 'kjeliasen/AdventOfCode'
# repo = 'Harleqin/advent-of-code-2017'
# repo = 'Bogdanp/awesome-advent-of-code'
# reporesp = zaq.github_api_request(f"https://api.github.com/repos/{repo}")


In [61]:
# reporesp.json()

In [62]:
git_json.language.value_counts().head()

Python        29
Haskell       26
JavaScript    19
Rust          11
Clojure        9
Name: language, dtype: int64

In [63]:
git_json[git_json.readme_contents == '13']

,language,language_all,readme_contents,repo_full,author,repo


In [64]:
git_json['readme'] = git_json.readme_contents.str.lower()

git_json['readme'] = [git_json.readme[i].replace(git_json.author[i].lower(), 'username') for i in range(0,len(git_json))]
git_json['readme'] = [git_json.readme[i].replace(git_json.repo[i].lower(), 'reponame') for i in range(0,len(git_json))]

# git_json.drop(columns='readme_contents')
# print(f'has author: {git_json.name_in_readme.sum()}\nhas repo{git_json.repo_in_readme.sum()}')
git_json.head()

,language,language_all,readme_contents,repo_full,author,repo,readme
0,Haskell,{'Haskell': 4468},# AdventOfCode18\nSolutions to some of the adv...,BartoszMilewski/AdventOfCode18,BartoszMilewski,AdventOfCode18,# reponame\nsolutions to some of the advent of...
1,Haskell,{'Haskell': 139842},Advent of Code 2019\n=====================\n\n...,mstksg/advent-of-code-2019,mstksg,advent-of-code-2019,advent of code 2019\n=====================\n\n...
2,Perl,"{'Perl': 138429, 'Awk': 60552, 'Shell': 59915,...",# Advent of Code\n\nMy solutions to [Advent of...,bewuethr/advent-of-code,bewuethr,advent-of-code,# advent of code\n\nmy solutions to [advent of...
3,F#,{'F#': 226573},# Advent of Code\n\nThese are my solutions in ...,CameronAavik/AdventOfCode,CameronAavik,AdventOfCode,# advent of code\n\nthese are my solutions in ...
4,Haskell,{'Haskell': 14869},# AofC2017\nAdvent of Code 2017\n,BartoszMilewski/AofC2017,BartoszMilewski,AofC2017,# reponame\nadvent of code 2017\n


In [67]:
git_json.drop(columns=['readme_contents']).to_json('datafiles/outdata.json', orient = 'columns')

In [68]:
json_chk = pd.read_json('datafiles/outdata.json')
json_chk.head()

,language,language_all,repo_full,author,repo,readme
0,Haskell,{'Haskell': 4468},BartoszMilewski/AdventOfCode18,BartoszMilewski,AdventOfCode18,# reponame\nsolutions to some of the advent of...
1,Haskell,{'Haskell': 139842},mstksg/advent-of-code-2019,mstksg,advent-of-code-2019,advent of code 2019\n=====================\n\n...
10,Kotlin,{'Kotlin': 53005},tginsberg/advent-2019-kotlin,tginsberg,advent-2019-kotlin,## advent of code 2019 solutions in kotlin\n\n...
100,Haskell,{'Haskell': 126402},mvaldesdeleon/aoc18,mvaldesdeleon,aoc18,"# advent of code 2018\n\nadvent of code 2018, ..."
101,Python,"{'Python': 156690, 'Rust': 104235, 'C++': 7148...",kodsnack/advent_of_code_2019,kodsnack,advent_of_code_2019,# advent of code 2019\ncontribute your solutio...
